In [ ]:
import os
import pandas as pd
import glob
import re

In [ ]:
def make_events_file(SST_stim_locations): 
    dfs = []
    for loc in SST_stim_locations:
        trial_type = loc.split("/")[-1].split(".")[0]
        df = pd.read_csv (loc, sep='\t+',header = None,engine='python')
        numrows_df = df.shape[0]
        df[2]=[trial_type for val in range(0,numrows_df)]
        dfs.append(df)
    all_events = pd.concat(dfs, ignore_index=True)
    all_events.set_axis(['onset', 'duration', 'trial_type'], axis=1,inplace=True)
    all_events.sort_values(by='onset', inplace=True, ascending=True)
    
    #this duplicates every row and puts duplicates one after the other
    all_events = pd.concat([all_events]*2)
    all_events.sort_values(by='onset', inplace=True, ascending=True)
    all_events.reset_index(drop=True, inplace=True)

    #this adds 0.2 to the onset of the first occurence of every row + _image to the title of these rows
    all_events.loc[0::2].onset +=0.2
    all_events.loc[0::2].trial_type+= '_image'
    
    #this adds 1.0 to the onset of the second occurence of every row + 0.5 to the duration + _arrow to the title of these rows
    all_events.loc[1::2].onset +=1
    all_events.loc[1::2].duration +=0.5
    all_events.loc[1::2].trial_type+= '_arrow'
    
    return all_events

In [ ]:
events_file_locations = (glob.glob("../../../sub-*/ses-*/func/*sst*run-*events.tsv"))
events_dict={}
for events_loc in events_file_locations:
    split_loc = events_loc.split('/')
    grp, num = re.split('(\d+)', split_loc[3].split("-")[-1])[0:2]
    ses = split_loc[4].split('-')[-1]
    run = split_loc[-1].split('_')[-2].split('-')[-1]
    if run != '3':
        if (grp+num+ses+run not in events_dict.keys()):
            SST_stim_locations = (glob.glob("../../../sourcedata/behavioral/{}_{}/{}/SST_R{}/*.txt".format(grp,num,ses,run)))
            all_events = make_events_file(SST_stim_locations)
            events_dict[grp+num+ses+run]=all_events
        else:
            all_events = events_dict[grp+num+ses+run]
        all_events.to_csv(events_loc, index = False, header=True, sep='\t', float_format='%.3f')
        